## Design for Pachira contracts

In [1]:
import os
import json
import subprocess
from termcolor import colored
from pprint import pprint
from uniswappy import *
from pachira import *

In [2]:
user_nm = 'user_machine_check'
tkn0_amount = 1000000000000000000000000
tkn1_amount = 1000000000000000000000000
token0Addr = '0x01'
token1Addr = '0x02'
basePoolAddr = '0x05'

In [3]:
tkn0 = ERC20("TKN0",token0Addr)
tkn1 = ERC20("TKN1", token1Addr)
exchg_data = UniswapExchangeData(tkn0 = tkn0, tkn1 = tkn1, symbol="LP", address="0x011", precision = UniswapExchangeData.TYPE_GWEI)

factory = UniswapFactory("Base pool factory", basePoolAddr)
base_lp = factory.deploy(exchg_data)
Join().apply(base_lp, user_nm, tkn0_amount, tkn1_amount)
base_lp.summary()

tkn2 = ERC20("TKN0", "0x09")
itkn1 = IndexERC20("iTKN1", "0x09", tkn1, base_lp)
exchg_data1 = UniswapExchangeData(tkn0 = tkn2, tkn1 = itkn1, symbol="LP1", address="0x012", precision = UniswapExchangeData.TYPE_GWEI)
child1_lp = factory.deploy(exchg_data1)
child1_lp.summary()

tkn3 = ERC20("TKN1", "0x09")
itkn2 = IndexERC20("iTKN0", "0x09", tkn1, base_lp)
exchg_data2 = UniswapExchangeData(tkn0 = itkn2, tkn1 = tkn3, symbol="LP2", address="0x012", precision = UniswapExchangeData.TYPE_GWEI)
child2_lp = factory.deploy(exchg_data2)
child2_lp.summary()

itkn3 = IndexERC20("iTKN0", "0x09", tkn0, base_lp)
itkn4 = IndexERC20("iTKN1", "0x09", tkn1, base_lp)
exchg_data3 = UniswapExchangeData(tkn0 = itkn3, tkn1 = itkn4, symbol="LP3", address="0x012", precision = UniswapExchangeData.TYPE_GWEI)
child3_lp = factory.deploy(exchg_data3)
child3_lp.summary()

Exchange TKN0-TKN1 (LP)
Reserves: TKN0 = 1000000000000000000000000, TKN1 = 1000000000000000000000000
Liquidity: 1000000000000000000000000 

Exchange TKN0-iTKN1 (LP1)
Reserves: TKN0 = 0, iTKN1 = 0
Liquidity: 0 

Exchange iTKN0-TKN1 (LP2)
Reserves: iTKN0 = 0, TKN1 = 0
Liquidity: 0 

Exchange iTKN0-iTKN1 (LP3)
Reserves: iTKN0 = 0, iTKN1 = 0
Liquidity: 0 



In [4]:
SETUP_amt = TokenDeltaModel(1000000)
SWAP_amt = TokenDeltaModel(10000)

lp0 = PachiraV1Exchange(base_lp)
lp1 = PachiraV1Exchange(child1_lp)
lp2 = PachiraV1Exchange(child2_lp)
lp3 = PachiraV1Exchange(child3_lp)

### Tree Setup

In [5]:
tree_base = TreeNode(lp0)
deploy_child = DeployChildNode(tree_base)

child1 = deploy_child.apply(lp1)
child2 = deploy_child.apply(lp2)
child3 = deploy_child.apply(lp3)

tree_base.summary()

Exchange TKN0-TKN1 (LP)
Reserves: TKN0 = 1000000000000000000000000, TKN1 = 1000000000000000000000000
Liquidity: 1000000000000000000000000 

Exchange TKN0-iTKN1 (LP1)
Reserves: TKN0 = 0, iTKN1 = 0
Liquidity: 0 

Exchange iTKN0-TKN1 (LP2)
Reserves: iTKN0 = 0, TKN1 = 0
Liquidity: 0 

Exchange iTKN0-iTKN1 (LP3)
Reserves: iTKN0 = 0, iTKN1 = 0
Liquidity: 0 



In [6]:
amount_in = UniV3Helper().dec2gwei(SETUP_amt.delta())

yield_amt1 = deploy_child.init_liquidity(child1, tkn0, user_nm, amount_in)
yield_amt2 = deploy_child.init_liquidity(child2, tkn1, user_nm, amount_in)
yield_amt31 = deploy_child.init_liquidity(child3, tkn0, user_nm, amount_in//2)
yield_amt32 = deploy_child.init_liquidity(child3, tkn1, user_nm, amount_in//2)

tree_base.summary()

Exchange TKN0-TKN1 (LP)
Reserves: TKN0 = 1196131224873762908863542, TKN1 = 1196131224873762908863541
Liquidity: 1195816568230055512631657 

Exchange TKN0-iTKN1 (LP1)
Reserves: TKN0 = 98065612436881450000000, iTKN1 = 47814324474041981688875
Liquidity: 68475842549052994146045 

Exchange iTKN0-TKN1 (LP2)
Reserves: iTKN0 = 50100534098950951717737, TKN1 = 98065612436881450000000
Liquidity: 70093791164613771525132 

Exchange iTKN0-iTKN1 (LP3)
Reserves: iTKN0 = 48950854828531289612523, iTKN1 = 48950854828531289612521
Liquidity: 48950854828531289612521 



### TreePair: Base-in TreeYield-out

In [7]:
tPair = TreePair(tree_base)
child_node = child1

In [8]:
amount_base_in = UniV3Helper().dec2gwei(SWAP_amt.delta())
recipient = user_nm
token_in = tkn1
token_out = child_node.lp.base_lp

child_node.parent.summary()
child_node.summary()

tree_yield = tPair.basein_tree_yieldout(child_node, token_in, token_out, recipient, amount_base_in)

child_node.parent.summary()
child_node.summary()

Exchange TKN0-TKN1 (LP)
Reserves: TKN0 = 1196131224873762908863542, TKN1 = 1196131224873762908863541
Liquidity: 1195816568230055512631657 

Exchange TKN0-iTKN1 (LP1)
Reserves: TKN0 = 98065612436881450000000, iTKN1 = 47814324474041981688875
Liquidity: 68475842549052994146045 

Exchange TKN0-TKN1 (LP)
Reserves: TKN0 = 1196131224873762908863542, TKN1 = 1196675235702482824191205
Liquidity: 1196088062710642365861619 

Exchange TKN0-iTKN1 (LP1)
Reserves: TKN0 = 98065612436881450000000, iTKN1 = 48085818954628834912976
Liquidity: 68669682047217982507293 



### TreePair: TreeYield-in Base-out

In [9]:
amount_yield_in = tree_yield
recipient = user_nm
token_in = child_node.lp.base_lp
token_out = tkn1

child_node.parent.summary()
child_node.summary()

base_amt_out = tPair.tree_yieldin_baseout(child_node, token_in, token_out, recipient, amount_yield_in)

child_node.parent.summary()
child_node.summary()

Exchange TKN0-TKN1 (LP)
Reserves: TKN0 = 1196131224873762908863542, TKN1 = 1196675235702482824191205
Liquidity: 1196088062710642365861619 

Exchange TKN0-iTKN1 (LP1)
Reserves: TKN0 = 98065612436881450000000, iTKN1 = 48085818954628834912976
Liquidity: 68669682047217982507293 

Exchange TKN0-TKN1 (LP)
Reserves: TKN0 = 1196131224873762908863542, TKN1 = 1196134476430894153471140
Liquidity: 1195817379271605164969715 

Exchange TKN0-iTKN1 (LP1)
Reserves: TKN0 = 98065612436881450000000, iTKN1 = 47815135515591634050389
Liquidity: 68475842549052994193693 



### Swaps on Tree

In [10]:
tree_in = tree_base

mm_lp = tree_in.lp
children = list(tree_base.child.values())
mm_lp1 = tree_in.child[children[0].name].lp
mm_lp2 = tree_in.child[children[1].name].lp
mm_lp3 = tree_in.child[children[2].name].lp

amount_in = UniV3Helper().dec2gwei(SWAP_amt.delta())
amount_out = mm_lp.exchangein_basein_baseout(tkn0, tkn1, amount_in, user_nm)

amount_in = UniV3Helper().dec2gwei(SWAP_amt.delta())
amount_out = mm_lp1.exchangein_basein_baseout(tkn0, itkn1, amount_in, user_nm)

amount_in = UniV3Helper().dec2gwei(SWAP_amt.delta())
amount_out = mm_lp2.exchangein_basein_baseout(itkn2, tkn3, amount_in, user_nm)

amount_in = UniV3Helper().dec2gwei(SWAP_amt.delta())
amount_out = mm_lp3.exchangein_basein_baseout(itkn4, itkn3, amount_in, user_nm)

### Final Tree State

In [11]:
tree_base.summary()

Exchange TKN0-TKN1 (LP)
Reserves: TKN0 = 1199034506058231340863542, TKN1 = 1193246885032704281220389
Liquidity: 1195817379271605164969715 

Exchange TKN0-iTKN1 (LP1)
Reserves: TKN0 = 100844614129610513500000, iTKN1 = 46501325532854179316318
Liquidity: 68475842549052994193693 

Exchange iTKN0-TKN1 (LP2)
Reserves: iTKN0 = 50362567397839808417737, TKN1 = 97556905915660063008104
Liquidity: 70093791164613771525132 

Exchange iTKN0-iTKN1 (LP3)
Reserves: iTKN0 = 48729996875171334956713, iTKN1 = 49173381351377498262521
Liquidity: 48950854828531289612521 

